In [2]:
# https://ppiazi.tistory.com/entry/python-cyclomatic-complexity-%EB%AA%A8%EB%93%88-lizard

In [3]:
# https://github.com/terryyin/lizard

In [4]:
# pip install cognitive_complexity

In [6]:
#customized preprocessing pkg
import preprocess as pp
import pandas as pd
import numpy as np
import pickle
import ast
from cognitive_complexity.api import get_cognitive_complexity
import pprint
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
# https://joaorafaelm.github.io/notebook/python/2021/04/10/cognitive-complexity.html

In [7]:
input_year = '2021'

### 소스코드 complexity 계산

In [ ]:
# 파일 불러오기
with open(f'../../data/complexity_python_df_{input_year}.pkl', 'rb') as f:
    complexity_python_df = pickle.load(f)

In [ ]:
complexity_python_df['cdate'].max()

In [ ]:
complexity_python_df['cdate'].max()

In [10]:
htmlp = pp.HTMLParser()
codep = pp.CodeSectionParser()

In [11]:
complexity_python_df['c_h_body'] = complexity_python_df['body'].apply(lambda x : htmlp.clean_html_str(x))

In [ ]:
complexity_python_df.info()

In [13]:
complexity_python_df['c_body'] = complexity_python_df['body'].apply(lambda x : codep(x))

In [ ]:
complexity_python_df

In [15]:
complexity_python_src = complexity_python_df[['id', 'c_body']]

In [ ]:
complexity_python_src['code_len'] = complexity_python_src['c_body'].apply(lambda x: len(x['code_sections']))

In [17]:
complexity_python_src = complexity_python_src[complexity_python_src['code_len']>0]

In [ ]:
complexity_python_src.head(5)

In [19]:
src_df = pd.DataFrame(columns = ['id', 'src'])
def code_explode(row) : 
    global src_df
    row['id']
    src_arr = []
    for i in range(row['code_len']):
        tmp_src_arr = []
        tmp_src_arr.append(row['id'])
        tmp_src_arr.append(row['c_body']['code_sections'][i]['span_str'])
        src_df = src_df.append(pd.Series(tmp_src_arr, index=src_df.columns), ignore_index=True)
    return True
        

In [ ]:
# bert_src_ex = bert_src[['q_id', 'q_prep_text']].apply(pd.Series.explode)
complexity_python_src.apply(code_explode, axis = 1)

In [21]:
src_df.reset_index(inplace=True)

In [ ]:
src_df.head()

In [24]:
for i in range(src_df.shape[0]):
    idx = src_df.iloc[i, 0]
    qid = src_df.iloc[i, 1]
    src = src_df.iloc[i, 2]
    file_nm = str(idx)+'_' + str(qid)+'.py'
    with open(f'../../data/src/{input_year}_src/{file_nm}', 'wb') as f:
        f.write(src.encode())


In [20]:
# https://github.com/rohaquinlop/complexipy

In [62]:
# pip install complexipy

In [ ]:
# !complexipy /home/mghan/sopjt/git/data/src/3_74963438.py -l file -o

In [28]:
# cyclomatic은 모든 언어 가능 but cognitive는 언어별로 따로 분석해야함
# 하려면 소나큐브 사야 함 
# pyhon 기준으로는 조금 삽질하면 할 수 있을 듯.... 해보고 유의미하면 소나큐브 고?


### complexity 결과 확인

In [11]:
path = f'../../data/src/{input_year}_excel/'
file_list = os.listdir(path)
file_list_csv = [file for file in file_list if file.endswith(".csv")]

# print ("file_list_csv: {}".format(file_list_csv))

In [12]:
calc_com_df = pd.DataFrame(columns = ['Path', 'File Name', 'Cognitive Complexity'])
for file_nm in file_list_csv :
    tmp = pd.read_csv(path+file_nm)
    calc_com_df = pd.concat([calc_com_df, tmp], axis = 0)

In [10]:
calc_com_df.head()

,Path,File Name,Cognitive Complexity


In [13]:
# 전체 데이터셋 
with open(f'../../data/calc_com_df_{input_year}.pkl', 'wb') as f:
    pickle.dump(calc_com_df, f)

In [ ]:
calc_com_df.head()

In [40]:
calc_com_df.reset_index(drop=True, inplace=True)

In [ ]:
calc_com_df.head()

In [42]:
calc_com_df['index'] = calc_com_df['Path'].apply(lambda x : x.split('_')[0])
# calc_com_df['Path'].apply(lambda x : [  for i in x.split('_')])

In [43]:
calc_com_df['id'] = calc_com_df['Path'].apply(lambda x : x.split('_')[1].split('.')[0])

In [ ]:
calc_com_df.sort_values(by = ['index', 'id'])

In [45]:
calc_com_df[['index', 'id', 'Cognitive Complexity']] = calc_com_df[['index', 'id', 'Cognitive Complexity']].astype('int')

In [46]:
src_df[['index', 'id']] = src_df[['index', 'id']].astype('int')

In [ ]:
calc_com_df.info()

In [ ]:
calc_com_df.sort_values(by = ['index', 'id'])

In [ ]:
calc_com_df.info()

In [ ]:
src_df.info()

In [51]:
calc_com_df = pd.merge(src_df,  calc_com_df, how = 'left', on = ['index', 'id'])

In [52]:
calc_com_df = pd.merge(complexity_python_df[['id', 'cdate']], calc_com_df, on = 'id')

In [ ]:
calc_com_df.head()

In [54]:
# 일자별로 아예 소스가 수행되지 않는 건 
calc_com_df_back = calc_com_df.copy()
calc_ed_com_df = calc_com_df[~calc_com_df['Path'].isna()]

In [55]:
calc_ed_com_df = calc_ed_com_df.sort_values(by = ['cdate', 'id', 'Cognitive Complexity'], ascending = [True, True, False]).groupby('id').head(1)

In [ ]:
calc_ed_com_df.head()

In [57]:
calc_ed_com_df = calc_ed_com_df.groupby('cdate').mean()[['Cognitive Complexity']].reset_index()

In [ ]:
calc_ed_com_df.head()

In [59]:
calc_ed_com_box_df = calc_ed_com_df.copy()

In [60]:
calc_ed_com_box_df['yyyymm'] = calc_ed_com_box_df['cdate'].str[:7]

In [61]:
ym_list = calc_ed_com_box_df['yyyymm'].unique()

In [ ]:
[calc_ed_com_box_df[calc_ed_com_box_df['yyyymm']==x]  for x in ym_list]

In [ ]:
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False

fig, axs = plt.subplots(1, 2, figsize = (20, 5), sharey=sharey, sharex=sharex)
# plt.xticks(rotation=45)
# major_xticks = ['2013-11-30','2014-11-30','2015-11-30','2016-11-30','2017-11-30','2018-11-30','2019-11-30','2020-11-30'
#                 ,'2021-11-30', '2022-11-30', '2023-11-30']
# yticks = np.arange(200, 2000, 400)
axs[0].bar(calc_ed_com_df['cdate']
        ,  calc_ed_com_df['Cognitive Complexity']
        ,  color = 'skyblue'
        # ,  edgecolor = 'black'
      ) # hist type

axs[0].set_title('Number of Questions over time', fontsize = 15)
axs[0].set_xlabel('Date', fontsize = 15)
axs[0].set_ylabel('# of Q', fontsize = 15)
# axs[0].set_xticks(major_xticks)
# axs[0].set_yticks(yticks)
# axs[0].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[0].tick_params(axis='x', rotation=45)
# axs[0].yticks(yt)

axs[0].bar(calc_ed_com_df['cdate']
        ,  calc_ed_com_df['Cognitive Complexity']
        ,  color = 'skyblue'
        # ,  edgecolor = 'black'
      ) # hist type

axs[1].set_title('Number of Answers over time', fontsize = 15)
axs[1].set_xlabel('Date', fontsize = 15)
axs[1].set_ylabel('# of A', fontsize = 15)
# axs[1].set_xticks(major_xticks)
# axs[1].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[1].tick_params(axis='x', rotation=45)

plt.show();


In [ ]:
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False

fig, axs = plt.subplots(1, 2, figsize = (20, 5), sharey=False, sharex=False)
# plt.xticks(rotation=45)
major_xticks = ['2022-01-30','2022-02-28','2022-03-30','2022-04-30','2022-05-30','2022-06-30','2022-07-30','2022-08-30','2022-09-30','2022-10-30','2022-11-30','2022-12-30',
                '2023-01-30','2023-02-28','2023-03-30','2023-04-30','2023-05-30','2023-06-30','2023-07-30','2023-08-30','2023-09-30','2023-10-30','2023-11-30','2023-12-30']
axs1_xticks = ym_list
axs1_xticks = np.insert(axs1_xticks, 0, '0')  # 인덱스 2 위치에 100을 추가 
# yticks = np.arange(200, 2000, 400)
axs[0].bar(calc_ed_com_df['cdate']
        ,  calc_ed_com_df['Cognitive Complexity']
        ,  color = 'skyblue'
        # ,  edgecolor = 'black'
      ) # hist type

axs[0].set_title('Avg Cognitive Complexity per Day', fontsize = 15)
axs[0].set_xlabel('Date', fontsize = 15)
axs[0].set_ylabel('Avg Cognitive Complexity', fontsize = 15)
axs[0].set_xticks(major_xticks)
# axs[0].set_yticks(yticks)
# axs[0].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[0].tick_params(axis='x', rotation=45)
# axs[0].yticks(yt)

axs[1].boxplot([calc_ed_com_box_df.loc[calc_ed_com_box_df['yyyymm']==x, 'Cognitive Complexity']  for x in ym_list]
        # ,  edgecolor = 'black'
      ) # hist type

axs[1].set_title('Cognitive Complexity per Month', fontsize = 15)
axs[1].set_xlabel('Date', fontsize = 15)
axs[1].set_ylabel('Cognitive Complexity', fontsize = 15)
axs[1].set_xticks(range(len(axs1_xticks)), axs1_xticks)
# axs[1].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[1].tick_params(axis='x', rotation=45)

plt.show();


In [ ]:
ym_list[:11]
# ym_list[12:]

In [ ]:
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False

fig, axs = plt.subplots(2, 1, figsize = (10, 10), sharey=True, sharex=False)
# plt.xticks(rotation=45)

axs0_xticks = ym_list[:11]
axs0_xticks = np.insert(axs0_xticks, 0, '0')  # 인덱스 2 위치에 100을 추가 

axs1_xticks = ym_list[12:]
axs1_xticks = np.insert(axs1_xticks, 0, '0')  # 인덱스 2 위치에 100을 추가 
yticks = np.arange(0.5, 5, 0.5)

axs[0].boxplot([calc_ed_com_box_df.loc[calc_ed_com_box_df['yyyymm']==x, 'Cognitive Complexity']  for x in ym_list[:11]]
        # ,  edgecolor = 'black'
      ) # hist type

axs[0].set_title('Cognitive Complexity per Month', fontsize = 15)
axs[0].set_xlabel('Date', fontsize = 15)
axs[0].set_ylabel('Cognitive Complexity', fontsize = 15)
axs[0].set_xticks(range(len(axs0_xticks)), axs0_xticks)
axs[0].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[0].tick_params(axis='x', rotation=45)


axs[1].boxplot([calc_ed_com_box_df.loc[calc_ed_com_box_df['yyyymm']==x, 'Cognitive Complexity']  for x in ym_list[12:]]
        # ,  edgecolor = 'black'
      ) # hist type

axs[1].set_title('Cognitive Complexity per Month', fontsize = 15)
axs[1].set_xlabel('Date', fontsize = 15)
axs[1].set_ylabel('Cognitive Complexity', fontsize = 15)
axs[1].set_xticks(range(len(axs1_xticks)), axs1_xticks)
axs[1].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[1].tick_params(axis='x', rotation=45)
plt.tight_layout()    
plt.show();


In [ ]:
len(np.arange(2, 13, 1))

In [ ]:
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False

fig, axs = plt.subplots(2, 1, figsize = (10, 10), sharey=True, sharex=False)
# plt.xticks(rotation=45)

axs0_xticks = ym_list[:11]
axs0_xticks = np.insert(axs0_xticks, 0, '0')  # 인덱스 2 위치에 100을 추가 

axs1_xticks = ym_list[12:]
axs1_xticks = np.insert(axs1_xticks, 0, '0')  # 인덱스 2 위치에 100을 추가 
yticks = np.arange(0.5, 5, 0.5)


axs[0].violinplot([calc_ed_com_box_df.loc[calc_ed_com_box_df['yyyymm']==x, 'Cognitive Complexity']  for x in ym_list[:11]], positions=np.arange(1, 12, 1))
axs[0].set_title('Cognitive Complexity per Month', fontsize = 15)
axs[0].set_xlabel('Date', fontsize = 15)
axs[0].set_ylabel('Cognitive Complexity', fontsize = 15)
axs[0].set_xticks(range(len(axs0_xticks)), axs0_xticks)
axs[0].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[0].tick_params(axis='x', rotation=45)



axs[1].violinplot([calc_ed_com_box_df.loc[calc_ed_com_box_df['yyyymm']==x, 'Cognitive Complexity']  for x in ym_list[12:]], positions=np.arange(1, 12, 1))
axs[1].set_title('Cognitive Complexity per Month', fontsize = 15)
axs[1].set_xlabel('Date', fontsize = 15)
axs[1].set_ylabel('Cognitive Complexity', fontsize = 15)
axs[1].set_xticks(range(len(axs1_xticks)), axs1_xticks)
axs[1].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[1].tick_params(axis='x', rotation=45)
plt.tight_layout()    
plt.show();
